# Spark

Hoewel het MapReduce algoritme van Hadoop een aantal voordelen heeft. 
De meest beperkende eigenschap van het MapReduce algoritme is de snelheid.
Omdat alles ingelezen wordt vanaf de harde schijf, tussenresultaten op de schijf opgeslagen worden en de finale resultaten ook wordt er tot wel 90% van de rekentijd gespendeerd in lees- of schrijfopdrachten.

Spark is geintroduceerd om dit te versnellen door gebruik te maken van in-memory processing.
Hierdoor is Spark tot 3 keer sneller op grote datasets en tot 100 keer op kleinere datasets.

Het spark framework kan gebruik maken van een externe opslag-locatie voor bestanden bij te houden (zoals HDFS) en bestaat uit de volgende componenten:
* SparkCore
* Spark SQL
* Spark Streaming
* MLlib
* SparkGraph

Daarnaast zijn er ook verschillende Spark Api's voor verschillende programmeertalen zoals Python, Scala, Java, ...
Hierdoor is het framework ook flexibeler dan het standaard MapReduce algoritme.
Heel veel informatie over het spark framework vind je in de [documentatie](https://spark.apache.org/docs/latest/quick-start.html) en de programming guides (bovenaan).

In [33]:
import pydoop.hdfs as hdfs

In [34]:
localFS = hdfs.hdfs(host='')
client = hdfs.hdfs(host='localhost', port=9000)

if not client.exists('/user/bigdata/06_Spark'):
    client.create_directory('/user/bigdata/06_Spark')
client.set_working_directory('/user/bigdata/06_Spark')
print(client.working_directory())

# do some cleaning in case anything else than input is present on HDFS
for f in client.list_directory("."):
    if not f["name"].endswith("input.txt"):
        client.delete(f["name"], True)
        
# upload input.txt
localFS.copy("input.txt", client, "input.txt")

2022-03-17 09:44:56,032 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


/user/bigdata/06_Spark


0

## Installatie

Een python implementatie van Spark kan eenvoudig geinstalleerd worden door het volgende commando uit te voeren. 
Dit moet maar eenmalig gebeuren.
Om te kijken of het reeds geinstalleerd is kan je kijken naar de versie van pyspark (indien geinstalleerd). 
Als de versie correct gereturned wordt, dan is het reeds geinstalleerd.

In [ ]:
!pip install pyspark

In [3]:
!pyspark --version

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 3.2.0
      /_/
                        
Using Scala version 2.12.15, OpenJDK 64-Bit Server VM, 11.0.14
Branch HEAD
Compiled by user ubuntu on 2021-10-06T12:46:30Z
Revision 5d45a415f3a29898d92380380cfd82bfc7f579ea
Url https://github.com/apache/spark
Type --help for more information.


Spark kan op drie manieren werken:
* Boven op MapReduce (traag)
* Boven op Yarn
* Via zijn eigen resource manager

In deze notebook gaan we gebruik maken van Spark gebruikmakende van yarn.   

## Resilient Distributed Datasets



In [2]:
from pyspark import SparkContext, SparkConf
# Spark Context -> die bevat informatie over waar de code moet uitgevoerd worden
# Spark Conf -> configuratie van de applicatie / metadata / naam /aantal cores

Voor de configuratie moeten we vooral twee zaken aangeven, namelijk:
* Naam van de applicatie (is zichtbaar in de yarn)
* Master url. De url dat het type cluster en hoe het te bereiken aangeeft. Wij gaan vooral werken met local om te communiceren met het lokale bestandssysteem 

In [3]:
conf = SparkConf().setAppName("wordcount").setMaster("yarn")
sc = SparkContext(conf=conf)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2022-03-17 08:41:52,794 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2022-03-17 08:42:09,586 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [4]:
# andere manier om de sparkcontext aan te maken
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
# get de sparkcontext als hij reeds bestaat, anders maak je een nieuwe aan
# er kan maar 1 sparkcontext tegelijkertijd actief zijn

## Wordcount voorbeeld

Om de api van pyspark te leren kennen kan je gaan naar de [documentatie](https://spark.apache.org/docs/latest/api/python/reference/index.html).
Een eerder stap bij stap uitleg kan je [hier](https://spark.apache.org/docs/latest/api/python/getting_started/index.html) vinden.

In onderstaande code gaan we stap voor stap het wordcount-voorbeeld uitwerken.

Eerst moet er een pyspark context aangemaakt worden als volgt.

In [8]:
# text file inlezen (dit is een dataframe)
textFile = spark.read.text("06_Spark/input.txt")

5


In [10]:
# aantal rijen
print(textFile.count())

# geef de eerste rij
print(textFile.first())

# geef het aantal rijen waarin het woord world voorkomt
print(textFile.filter(textFile.value.contains("world")).count())

5


Row(value='Hello World,')
2


In [24]:
# wordcount example
# splits elke lijn in woorden
# sc.textFile -> geeft een rdd
words = sc.textFile("06_Spark/input.txt").flatMap(lambda line: line.split(" "))
# zet elk woord om naar (woord, 1)
wordcounts1 = words.map(lambda word: (word[0], 1) if len(word) > 0 else ("",1))
# tel alle values op in het woord
# wordt gedaan per woord afzonderlijk
# a is de running count
# (woord, 1)
# (woord, 1)
# a=0 , b=1 (value van het eerste woord) -> a+b = nieuwe a
# a=1 , b=1 (value van het eerste woord) -> a+b = nieuwe a
wordcounts2 = wordcounts1.reduceByKey(lambda a,b: a+b)
wordcounts2.saveAsTextFile("06_Spark/output3.txt")

**Wat gebeurt er in dit voorbeeld?**

Sparkcontext om een connectie te maken met de distributed storage
De input file wordt dan ingelezen met de textFile functie.
Door middel van de flatMap functie wordt de tekst lijn per lijn ingelezen en gesplits in woorden. 
Dit resulteert in een RDD (Resilient Distributed Dataset.
De .map() functie maakt een key-value pair aan voor elke keer dat het woord voorkomt.
In een laatste fase is er een reduce stap per key die de som neemt van alle keren dat het woord voorkomt om de uiteindelijke wordcount te nemen.
Of af te ronden wordt het resultaat opgeslagen.

![spark wordcount in yarn](images/yarn_001.png)

## SparkSession

Nu gaan we stuk voor stuk de verschillende stappen bekijken om een pyspark applicatie te maken.
De eerste stap is het aanmaken van een sessie (SparkSession) wat het beginpunt is voor spark applications.
Er zijn twee manieren om een SparkSession aan te maken:
* builder()
* newSession()

Bij het aanmaken van een session wordt er intern een SparkContext object aangemaakt. 
Dit object stelt de connectie naar een cluster voor.
Er kan maar 1 context tegelijkertijd actief zijn.
Als je wil connecteren met een tweede cluster moet je eerst stop() oproepen op de reeds actieve context.

In [5]:
# draai spark lokaal (niet op een cluster)
# 1 omdat we 1 core willen toekennen
spark = SparkSession.builder.master("local[1]").appName("Spark_les").getOrCreate()

#spark.stop()

# zonder argument in master draait op de cluster
# hier kiezen we voor 2 cores
spark2 = SparkSession.builder.config("spark.driver.cores", 2).appName("Spark_les").getOrCreate()

## RDD

Op basis van het SparkSession object is het dan mogelijk om RDD-objecten aan te maken.
Een RDD is de basis dataobject binnen Spark dat in parallel op verschillende nodes binnen een cluster kan uitgevoerd worden.
Alle dataobjecten binnen spark horen tot deze klassen en dus zijn er veel mogelijkheden om RDD's aan te maken.
Hier haal ik er twee aan:
* parallelize() om bestaande python objecten om te zetten naar een RDD
* textFile() of andere read methoden om bestanden op de cluster uit te lezen

In [76]:
# voorbeeld parallelize
dataList = [("Student1", 8), ("Student2", 16), ("Student3", 11)]
rdd = spark.sparkContext.parallelize(dataList)

# voorbeeld textFile
rdd2 = spark.sparkContext.textFile("06_Spark/input.txt")

Met bovenstaande methoden hebben we twee rdd's aangemaakt. 
Op deze objecten kunnen nu verscheidene operaties uitgevoerd worden.
Een belangrijke eigenschap van dit type objecten is dat ze steeds in parallel uitgevoerd worden.

De beschikbare operaties kunnen in twee groepen verdeeld worden:
* transformaties
* acties

[Transformaties](https://sparkbyexamples.com/apache-spark-rdd/spark-rdd-transformations/) zijn lazy-operations waarvoor de berekening uitgesteld wordt en geven een nieuw RDD terug.
Een aantal voorbeelden van transformaties zijn:
* flatMap()
* map()
* reduceByKey()
* filter()
* sortByKey()

[Acties](https://sparkbyexamples.com/apache-spark-rdd/spark-rdd-actions/) zijn operaties die een berekening starten (ook van de nodige transformaties) en geven een niet RDD-object terug. 
Een aantal voorbeelden hiervan zijn:
* count()
* collect()
* first()
* max()
* reduce()

Lees nu bovenstaande links en geef de functies die nodig zijn voor de volgende vragen op te lossen. Geef ook aan of het transformaties zijn of acties:
* Het aantal keer dat elke waarde aanwezig is in de dataset (1 functie voor wordcount uit te voeren)
* Uitfilteren van rijen
* Groeperen van een aantal rijen op basis van een bepaalde waarde.
* Toevoegen van een kolom aan elke key (bvb de lengte van een woord)
* Hoe doe je head() uit pandas op RDD's?
* Hoe doe je de apply() uit pandas op RDD's?

Maak nu een spark applicatie dat van de eerste RDD (met de studenten) telt hoeveel studenten geslaagd zijn.

In [7]:
rdd.filter(lambda student: student[1] >= 10).count()

2

De applicatie voor het berekenen van een gemiddelde is iets complexer.
Dit soort applicaties kan geschreven worden als volgt:

In [8]:
# aantal elementen in rdd
aantal = rdd.count()

# elk element delen door dit aantal
# iets doen voor elk element kan door de map() en reduce()
rdd.map(lambda x: x[1]).reduce(lambda x,y: x+y) / aantal

11.666666666666666

Schrijf nu een mapreduce applicatie in spark om de tweede RDD van de input te verwerken en het aantal woorden van elke lengte te bekomen.

In [9]:
print(rdd2.collect())
# map doet de lambda per element
# flatMap doet een map maar zet dan alles in 1 lijst
words = rdd2.flatMap(lambda line: line.split(" "))
print(words.collect())
wordcounts1 = words.map(lambda word: (len(word), 1))
print(wordcounts1.collect())
result = wordcounts1.reduceByKey(lambda a,b: a+b)
print(result.collect())

['Hello World,', 'hello world,', 'hello world,', '', 'Dit is een voorbeeld file om het Wordcount voorbeeld te testen !']


['Hello', 'World,', 'hello', 'world,', 'hello', 'world,', '', 'Dit', 'is', 'een', 'voorbeeld', 'file', 'om', 'het', 'Wordcount', 'voorbeeld', 'te', 'testen', '!']


[(5, 1), (6, 1), (5, 1), (6, 1), (5, 1), (6, 1), (0, 1), (3, 1), (2, 1), (3, 1), (9, 1), (4, 1), (2, 1), (3, 1), (9, 1), (9, 1), (2, 1), (6, 1), (1, 1)]


[(6, 4), (0, 1), (2, 3), (4, 1), (5, 3), (3, 3), (9, 3), (1, 1)]


## Dataframes

Een belangrijke subklasse van RDD's zijn dataframes.
Dit is een veel gebruikte manier om gestructureerde data voor te stellen.
Dataframes in spark is sterk gerelateerd aan de dataframes gezien in pandas.
Het belangrijskte verschil is dat ze verdeeld worden over de cluster en operaties op de dataframes in parallel uitgevoerd worden.
Dataframes kunnen aangemaakt worden door gebruik te maken van de createDataFrame functie in context of ingelezen worden vanuit csv's of jsons. Ten slotte kunnen dataframes ook komen van externe bronnen zoals databases als resultaat van een sql-query.

In [21]:
data = [('Harry', 'Potter','1980-07-31','M',100000000),
  ('Ronald','Wemel','1980-04-01','M',10),
  ('Hermelijn','Griffel','1979-09-19','F',4000)
]

columns = ["firstname", "lastname", "dob", "gender", "budget"]
df = spark.createDataFrame(data = data, schema=columns)
df.show()

df.printSchema()
# dit gaat het nog niet tonen
# dit staat op de cluster, niet op de lokale machine
df.describe()
# dataframe lokaal hebben
df.collect()
# enkel printen
df.show()

+---------+--------+----------+------+---------+
|firstname|lastname|       dob|gender|   budget|
+---------+--------+----------+------+---------+
|    Harry|  Potter|1980-07-31|     M|100000000|
|   Ronald|   Wemel|1980-04-01|     M|       10|
|Hermelijn| Griffel|1979-09-19|     F|     4000|
+---------+--------+----------+------+---------+

root
 |-- firstname: string (nullable = true)
 |-- lastname: string (nullable = true)
 |-- dob: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- budget: long (nullable = true)



+---------+--------+----------+------+---------+
|firstname|lastname|       dob|gender|   budget|
+---------+--------+----------+------+---------+
|    Harry|  Potter|1980-07-31|     M|100000000|
|   Ronald|   Wemel|1980-04-01|     M|       10|
|Hermelijn| Griffel|1979-09-19|     F|     4000|
+---------+--------+----------+------+---------+



## PySpark SQL

Bovenstaande datastructuren (RDD's en Dataframes) zijn een onderdeel van het Pyspark sql module.
De Spark API heeft een hele reeks methoden en functies om deze in te laden, uit te lezen en te manipuleren.
Daarnaast maakt deze module het ook mogelijk om SQL-queries uit te voeren op dataframes.
Om SQL-queries uit te voeren op dataframes moet er eerst een view gemaakt worden in het dataframe met de functie createOrReplaceTempView("view_name")

Daarna kan je gebruik maken van de .sql() functie om allerhande sql queries uit te voeren.

In [30]:
# om sql queries uit te kunnnen voeren (maak een tabel test aan die het dataframe bijhoudt)
df.createOrReplaceTempView("test")
df_male = spark.sql("select * from test where gender='M'")
df_male.show()

# createOrReplaceTempView is hier niet voor nodig
df_male2 = df.filter(df.gender == "M")
df_male2.show()

# groupby zaken tonen -> aantal van elk geslacht
df_num_gender = spark.sql("select gender,count(*) from test group by gender")
df_num_gender.show()

df_num_gender2 = df.groupby("gender").count()
df_num_gender2.show()

# kolommen selecteren, of zaken gaan berekenen
df.select("firstname", "lastname").show()

+---------+--------+----------+------+---------+
|firstname|lastname|       dob|gender|   budget|
+---------+--------+----------+------+---------+
|    Harry|  Potter|1980-07-31|     M|100000000|
|   Ronald|   Wemel|1980-04-01|     M|       10|
+---------+--------+----------+------+---------+

+---------+--------+----------+------+---------+
|firstname|lastname|       dob|gender|   budget|
+---------+--------+----------+------+---------+
|    Harry|  Potter|1980-07-31|     M|100000000|
|   Ronald|   Wemel|1980-04-01|     M|       10|
+---------+--------+----------+------+---------+



+------+--------+
|gender|count(1)|
+------+--------+
|     M|       2|
|     F|       1|
+------+--------+



+------+-----+
|gender|count|
+------+-----+
|     M|    2|
|     F|    1|
+------+-----+

+---------+--------+
|firstname|lastname|
+---------+--------+
|    Harry|  Potter|
|   Ronald|   Wemel|
|Hermelijn| Griffel|
+---------+--------+



Buiten de functionaliteit om SQL queries uit te voeren is ook het lezen en schrijven van allerhande dataformaten een belangrijk onderdeel van de pyspark sql module.
Meer informatie hierover kun je [hier](https://spark.apache.org/docs/latest/sql-data-sources.html) vinden in de documentatie.
In essentie ziet de code er uit als volgt:

In [ ]:
df = spark.read.option("delimiter", ";").option("header",True).option("multiline", True).csv("{}")

De opties die hierbij gekozen kunnen worden kun je vinden in de documentatie.

Daarnaast zijn er ook functionaliteiten om data uit te lezen speciaal voor Machine Learning zoals libsvm en image-directories maar die worden later getoond.

### Oefening

Net zoals RDD kunnen er een aantal operaties uitgevoerd worden op deze dataframes.
Een volledige lijst met alle operaties kan je [hier](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql.html#dataframe-apis) vinden.
Zoek de functies die gebruikt moeten worden om de volgende zaken uit te voeren:
* Groepeer volgens een bepaalde sleutel
* Krijg een lijst met alle kolomnamen
* Filter rijen uit
* Verwijder null-values in de dataset via rijen
* Verwijder null-values door kolommen te verwijderen
* Bereken een dataframe met statistieken van het dataframe
* Krijg een dataframe met alle nan waarden
* Hoe krijg je informatie zoals .info()
* Hoe werkt het groeperen van informatie op basis van een key/kolom

Probeer deze ook uit op bovenstaand aangemaakt dataframe

In [ ]:
# groupby
# .columns
# filter()     / where() (alias)
# dropna()
# drop() voor de kolommen (dit gaat manueel moeten gedaan worden -> kijk eerst of er kolommen zijn met veel null-waarden)
# describe()
# isnan() -> eventueel een kolomnaam in 
   # -> is dit hetzelfde als isnull() 
       # null = geen data
       # nan = data is geen nummer -> "Harry" in numerieke kolom
# printSchema()/.schema
# groupby() -> is een transformatie (gaat het df gaan verdelen in groepjes) wordt pas uitgevoerd als er een actie is
    # een actie is -> count(), max(), ... (iets dat de groepen gaat samenvoegen tot 1 rij)

Lees daarna volgende [link](https://sparkbyexamples.com/pyspark/pyspark-aggregate-functions/) om een idee te krijgen over hoe verschillende functies uit te voeren op deze dataframes.
Werk nu de volgende oefening uit en maak hiervoor een spark applicatie:
* Laad de kleine dataset over soorten irissen uit via sklearn en slap deze op als csv
* Upload de csv naar de cluster
* Schrijf de code om de csv uit te lezen en om te zetten naar een dataframe
* Print het dataschema uit voor het dataframe en bepaal hoeveel kolommen er aanwezig zijn in deze dataset
* Bereken de gemiddelde waarde van elke kolom
* Bereken hoeveel waarden in elke kolom groter zijn dan het gemiddelde
* Voeg een kolom toe dat bijhoudt hoeveel waarden op elke rij groter zijn dan het gemiddelde van die kolom
* Selecteer de rijen waar het target/klasse/label gelijk is aan 2

In [31]:
from sklearn.datasets import load_iris
import pandas as pd

df = load_iris(as_frame=True).frame
df.to_csv("input.csv")

In [35]:
if not client.exists('/user/bigdata/06_Spark/demo'):
    client.create_directory('/user/bigdata/06_Spark/demo')
client.set_working_directory('/user/bigdata/06_Spark/demo')

# do some cleaning in case anything else than input is present on HDFS
for f in client.list_directory("."):
    if not f["name"].endswith("input.csv"):
        client.delete(f["name"], True)
        
# upload input.txt
localFS.copy("input.csv", client, "input.csv")

0

In [73]:
#%%file spark_demo.py
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, col, sum, when, count

# eerst een spark sessie aanmaken
spark = SparkSession.builder.config("spark.driver.cores", 2).appName("oefening_spark_les").getOrCreate()

# lees csv
df = spark.read.option("delimiter", ",").option("header", True).csv("/user/bigdata/06_Spark/demo/input.csv")
# df.printSchema()
df = df.drop("_c0") # inplace gaat niet, alles maakt altijd een nieuw dataframe aan

# bereken het gemiddel van elke kolom
#df.select(avg("sepal length (cm)")).show()
#df.select(avg("sepal length (cm)"), avg("sepal width (cm)")).show()

cols = ["sepal length (cm)", "sepal width (cm)"]
#df.select([avg(c) for c in cols]).show()
df_avg = df.select([avg(c).alias(c) for c in df.columns]).collect()[0]

# aantal waarden groter dan het gemiddelde in die kolom
# 1. op zoek gaan naar welke waarden groter dan het gemiddelde
# 2. tel al deze op

# df_avg[0] is het gemiddelde van de eerste kolom
df_larger_than_avg = df.select([(col(c) > df_avg[index]).alias(c) for index, c in enumerate(df.columns)])
#df_larger_than_avg.select([sum(col(c).cast("int")) for c in df_larger_than_avg.columns]).show()
#df_larger_than_avg.select([count(when(col(c), 1)) for c in df_larger_than_avg.columns]).show()

# extra kolom met het aantal op die rij groter dan het gemiddelde
# kolom toevoegen doe je met withColumn("naam", hoe ze te bereken)
tmp = df_larger_than_avg.select([col(c).cast("int").alias(c) for c in df.columns])
cols = tmp.columns
df2 = tmp.withColumn("aantal", col(cols[0]) + col(cols[1]) + col(cols[2]) + col(cols[3]) + col(cols[4]))
#df2.show(5)

# target label is 2
df.filter(df.target == 2).show()
df.withColumn("klasse", when(col("target") == 0, "klasse 0")
              .when(col("target") == 1, "klasse 1")
             .otherwise("klasse 2")).show(5)

+-----------------+----------------+-----------------+----------------+------+
|sepal length (cm)|sepal width (cm)|petal length (cm)|petal width (cm)|target|
+-----------------+----------------+-----------------+----------------+------+
|              6.3|             3.3|              6.0|             2.5|     2|
|              5.8|             2.7|              5.1|             1.9|     2|
|              7.1|             3.0|              5.9|             2.1|     2|
|              6.3|             2.9|              5.6|             1.8|     2|
|              6.5|             3.0|              5.8|             2.2|     2|
|              7.6|             3.0|              6.6|             2.1|     2|
|              4.9|             2.5|              4.5|             1.7|     2|
|              7.3|             2.9|              6.3|             1.8|     2|
|              6.7|             2.5|              5.8|             1.8|     2|
|              7.2|             3.6|              6.

[Row(sepal length (cm)='6.7', sepal width (cm)='3.0', petal length (cm)='5.2', petal width (cm)='2.3', target='2', klasse='klasse 2'),
 Row(sepal length (cm)='6.3', sepal width (cm)='2.5', petal length (cm)='5.0', petal width (cm)='1.9', target='2', klasse='klasse 2'),
 Row(sepal length (cm)='6.5', sepal width (cm)='3.0', petal length (cm)='5.2', petal width (cm)='2.0', target='2', klasse='klasse 2'),
 Row(sepal length (cm)='6.2', sepal width (cm)='3.4', petal length (cm)='5.4', petal width (cm)='2.3', target='2', klasse='klasse 2'),
 Row(sepal length (cm)='5.9', sepal width (cm)='3.0', petal length (cm)='5.1', petal width (cm)='1.8', target='2', klasse='klasse 2')]

In [40]:
#!python spark_demo.py

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2022-03-17 09:49:40,994 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2022-03-17 09:49:48,758 WARN util.Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
2022-03-17 09:51:05,827 WARN csv.CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , sepal length (cm), sepal width (cm), petal length (cm), petal width (cm), target
 Schema: _c0, sepal length (cm), sepal width (cm), petal length (cm), petal width (cm), target
Expected: _c0 but found: 
CSV file: hdfs://localhost:9000/user/bigdata/06_Spark/demo/input.csv
+---+-----------------+----------------+-----------------+----------------+------+
|_c0|sepal length (cm)|sepal width (cm)|petal length (cm)|petal width (cm)|target|
+---+-----------------+----------------+-----------------+------

**Shared variabelen**

Variabelen met read-write acces zijn zeer inefficient om te gebruiken in een cluster met sterke parallelisatie.
Spark bied echter twee varianten aan die wel efficient geimplementeerd kunnen worden, namelijk
* Broadcasted variabelen
* Accumulators

Broadcasted variabelen zijn read-only variabelen, die aangemaakt worden door de driver en eenmalig verspreid worden over de nodes in plaats van voor elke job.
Dit wordt vooral gebruikt om grote data die veelvuldig gebruikt wordt te cachen op de nodes.
Bij het gebruik van broadcast variabelen is het belangrijk om te onthouden dat je de originele variabele niet meer mag gebruiken na het aanmaken van de broadcasted variabele omdat ze anders toch nog elke job doorgestuurd wordt.
De belangrijkste functies om te werken met broadcasted variabelen zijn:

In [75]:
broadcasted = spark.sparkContext.broadcast("dit is een broadcasted string")
broadcasted.value
broadcasted.unpersist() # tijdelijke delete -> kan opnieuw doorgetuurd worden wanneer nodig
broadcasted.destroy() # permanente delete, kan niet meer gebruikt worden

'dit is een broadcasted string'

**Accumulators**

Het andere type dat aangeboden wordt zijn accumulators.
Deze laten enkel toe dat noden iets toevoegen aan een gedeelde variabele.
Enkel de driver kan deze variabele uitlezen.
Dit kan bijvoorbeeld gebruikt worden om tellers of sommen bij te houden.
Deze accumulators kunnen een naam hebben (named accumulators zijn zichtbaar in de wep api).
De ingebouwde accumulator van Spark ondersteunt enkel numerieke accumulators.
Het is echter mogelijk om eigen accumulators toe te voegen door over te erven van de AccumulatorParam klasse en deze twee functies te implementeren:
* zero: De begin waarde van de accumulator
* addInPlace: Om twee waarden samen te voegen

In [79]:
#dataList = [("Student1", 8), ("Student2", 16), ("Student3", 11)]

# numeriek
accumVar = spark.sparkContext.accumulator(0)
rdd.foreach(lambda x: accumVar.add(x[1]))
print(accumVar.value)

# find max score
from pyspark.accumulators import AccumulatorParam
class MaxAccumulator(AccumulatorParam):
    def zero(self, value):
         return value
    
    def addInPlace(self, var, new_value):
        if var < new_value:
            return new_value
        return var
    
accumVar = spark.sparkContext.accumulator(0, MaxAccumulator())
rdd.foreach(lambda x: accumVar.add(x[1]))
print(accumVar.value)

# add to list
class ListAccumulator(AccumulatorParam):
    def zero(self, value):
        return []
    
    def addInPlace(self, var, new_value):
        var.extend(new_value)
        return var
    
# note that to store a list, lists must be passed to initial value and add
accumVar = spark.sparkContext.accumulator([], ListAccumulator())
rdd.foreach(lambda x: accumVar.add([x[1]]))
print(accumVar.value)

35
16
[8, 16, 11]


## Pandas

Door de hoge populariteit van pandas in python is er een alternatief uitgewerkt binnen de laatste versie van Spark (eind 2021) dat de pandas api integreert.
Hierdoor kan je code schrijven die identiek is aan te werken met pandas.
Lees [dit artikel](https://towardsdatascience.com/run-pandas-as-fast-as-spark-f5eefe780c45) om meer informatie te krijgen over de verschillen tussen de dataframes API en de pandas-on-spark API.
De documentatie voor deze api vind je [hier](https://spark.apache.org/docs/latest/api/python/user_guide/pandas_on_spark/index.html).
Een belangrijk onderdeel van deze documentatie omvat de best practices:
* Check execution plan (dmv de explain() functie)
* Use checkpoints voor fout-tolerantie en efficientie van de planner.
 * df.spark.local_checkpoint()
* Vermeid data shuffling (sorting) omdat hierbij data tussen nodes moet gestuurd worden wat niet efficient is.
* Vermeid berekeningen op 1 partitie (geen parallellisatie)
* Vermeid kolomnamen startend of eindigend op "_"
 * Deze worden gebruikt door interne functies van pandas/spark
* Kolomnamen moeten uniek zijn
* Specificeer de index kolom bij omzetten dataframe en pandas-on-spark API
* Gebruik zoveel mogelijk van de pandas-on-spark API direct ipv standaard python functies om conflicten te vermijden
 * df.sum() werkt maar sum(df) niet
* Vermeid operaties op meerdere dataframes want deze gebruiken een join intern en is hierdoor traag